# Analysis Ready Data Tutorial Part 1: Introduction and Best Practices

Time-series analysis (e.g. change detection and trend detection) is a powerful application of satellite imagery. However, a great deal of processing is required to prepare imagery for analysis. Analysis Ready Data (ARD), preprocessed time-series stacks of overhead imagery, allow for time-series analysis without any additional processing of the imagery. See [Analysis Data Defined](https://medium.com/planet-stories/analysis-ready-data-defined-5694f6f48815) for an excellent introduction and discussion on ARD.

This tutorial shows how [Planet APIs](https://developers.planet.com/docs/apis/) can simplify production of ARD by demonstrating best practices and then by walking through a real world use case. This tutorial is targeted to users who have little to no geospatial knowledge but have experience working with APIs. The goal of this tutorial is to teach the user the how and whys of using the Data and Orders APIs to create and interpret ARD for both use cases. This first part of the totorial focuses on best practices. The following part will focus on a real-world use case.


## APIs

The two Planet APIs that are used in creation of ARD are the [Data API](https://developers.planet.com/docs/data/) and the [Orders API](https://developers.planet.com/docs/orders/). The Data API is used to search for imagery according to certain criteria based on the use case, and the Orders API is used to process that imagery into ARD that can be fed directly into time-series analysis.

### Data API

The first step in using the Data API is identifying the search criteria. This is specifying answers to the following questions regarding the use case time-series analysis:
* What is the time range?
* What product item type is desired?
* What is the area of interest (geographic region)?
* What percentage of pixels need to be usable?
* etc.

While time range is likely pretty trivial to determine, product item, area of interest, and usable pixels may take a little bit of work. Let's dive into each further

#### Product Item Type

The [product item type](https://developers.planet.com/docs/data/items-assets/) refers to the sensor source (aka satellite type) and basic processing desired. This decision is highly dependent on application, as coverage, revisit rate, spectral bands, and resolution differ between products. A good overview of the products available in the Planet Data API is provided on the [Planet Imagery and Archive](https://www.planet.com/products/planet-imagery/) page (look for the link to product specs for details). For most frequent revisit rate, we will use the PS satellite. Experience has shown that customers most often use the scene (vs the orthotile) product. Therefore, this tutorial will focus on the `PSScene4Band` product.

#### Area of Interest

The area of interest is the geographic region for the analysis, given as GeoJSON. If you are familiar with JSON, the format of GeoJSON will likely be easy to grasp. According to <geojson.org>, "GeoJSON is a format for encoding a variety of geographic data structures." The specific geographic data structure we are interested is a `Polygon`. The [GeoJSON wikipedia page](https://en.wikipedia.org/wiki/GeoJSON) gives some great examples of the data structures for the various GeoJSON data structures. 

Some care needs to be given to describing the [position](https://tools.ietf.org/html/rfc7946#section-3.1.1) for each coordinate in a GeoJSON geometry. Each position is specified as `(longitude, latitude)` or `(easting, northing)` (order really matters here!). Also, this may be a surprise, but the same point on earth can have different `(longitude, latitude)` values based on the spatial reference system. Basically, a spatial reference system describes where something is in the real world. But there are thousands of different spatial reference systems. These are separated into two categories: geographic and projected. Geographic coordinate systems model the earth as an ellipsoid (this model is called the `datum`) and describe positions on the surface of the earth (coordinates) in terms of the prime meridian and angle of measure. Projected coordinate systems take this a step further by projecting the geographic coordinates (quite three-dimensional) into two dimensions (the `projection`). Different projections preserve different properties, such as area, angles, or direction for north. There is a rich area of discovery, discussion, and even a little [teasing](https://xkcd.com/977/) (thanks xkcd!) in the world of spatial reference systems.

GeoJSON only supports one spatial reference system, [WGS84](https://spatialreference.org/ref/epsg/wgs-84/). This is a geographic coordinate system describing locations in latitude, longitude. However, many web mapping applications use the Web Mercator projected coordinate system to to describe locations. Confusingly, Web Mercator also describes locations in latitude, longitude. But a `(longitude, latitude)` GeoJSON position given in Web Mercator will **not** end up where you expect if it is not first projected into WGS84. The easiest way to define an aoi that is described in WGS84 is to draw it in [Planet Explorer](https://www.planet.com/explorer) and click the little button "Download AOI as GeoJSON."

#### Usable Pixels

Taking pictures of the earth from space ain't easy. There are a lot of steps and a lot of things have to go right to get a clear shot. Therefore, not every pixel in every image taken from space is useful for analysis. For one, images taken from space are projected into a spatial reference system (discussed briefly above), a process that introduces some NoData (aka outside of the image footprint) pixels into the resulting image. Additionally, clouds cover a great deal of the earth and create cloudy pixels when imaged. While some applications can use cloudy pixels, others cannot. Therefore, the type of pixels that are determined to be 'usable' are often application-specific. To support definition of usable pixels, and filtering based on that definition, Planet provides Usable Data Masks along with Usable Data entries in the imagery metadata. For more details on the Usable Data Mask, check out [Clear for Analysis with Planet’s New Usable Data Masks](https://www.planet.com/pulse/planets-new-usable-data-masks/). You can also find great examples for working with Usable Data Masks in the [UDM2 notebooks](https://github.com/planetlabs/notebooks/tree/master/jupyter-notebooks/udm2). For more information on the Usable Data metadata entries, see [Usable Data in Planet imagery](https://developers.planet.com/planetschool/usable-data-in-planet-imagery/).

### Orders API

The core decision around using the orders api is which [product bundle](https://developers.planet.com/docs/orders/product-bundles-reference/) to use. This is the starting point for all processing and there are a lot of options. Once the product is determined, the processing steps (aka tools and toolchains) are defined. Finally, the logistics of the delivery of the imagery are ironed out.

#### Product Bundle

To enable time-series analysis, ARD imagery must be processed so that imagery is consistant across days, months, and possibly years. This means correcting for differences in camera sensitivities, the relative location of the sun, and the atmospheric conditions. The Analytic Radiance (`analytic`) product bundle provides imagery corrected for difference in camera sensitivities and location of the sun (as radiance) and can also remove the effect of the sun's spectrum by applying the `reflectanceCoefficient` value given in the imagery metadata. However, the Analytic Surface Reflectance (`analytic_sr`) product bundle removes the effect of atmospheric conditions while also converting to reflectance. Therefore, the Analytic Surface Reflectance is the ideal product for ARD and the one we will use here.

#### Tools and Toolchains

The [Tools and Toolchains](https://developers.planet.com/docs/orders/tools-toolchains/) functionality in the Orders API are the key to seamlessly creating ARD. Through the API, one can define the pre-processing steps for the data before it is delivered. Given proper definition of the tools and toolchains, data that is delivered is analysis-ready.

#### Delivery

There are a few options for delivery that cater to different use cases. Imagery can be downloaded directly or delivered to [cloud storage](https://developers.planet.com/docs/orders/ordering-delivery/#delivery-to-cloud-storage). When imagery is downloaded, the user can poll for when the order is ready or notifications ([e-mail](https://developers.planet.com/docs/orders/ordering-delivery/#email-notification) or [webhooks](https://developers.planet.com/docs/orders/ordering-delivery/#using-webhooks)) can be used. Additionally, the imagery can be delivered as a [zip archive](https://developers.planet.com/docs/orders/ordering-delivery/#zipping-results).

## Best Practices

Now that we have a basic understanding of the Data and Orders APIs, let's put them to use creating ARD. This will be a demonstration of best practices.

For this tutorial we will use the [planet python client](https://github.com/planetlabs/planet-client-python) ([documentation](https://planetlabs.github.io/planet-client-python/index.html)). This client simplifies interactions with the various Planet APIs and also includes a command-line interface.

The first step in using the planet python client is initializing the client with the user API key. Each user (or organization) has their own unique API key. Information on finding the API key and running Docker so that the API key is available in the notebooks is given in this repository's [README](https://github.com/planetlabs/notebooks#install-and-use-these-notebooks).

The next step is building functionality for searching the Data API with the client. This consists of building the search query and then running the search. In building this functionality, we will use test information for data ranges and AOIs to test the functionality, but we will build the functions so those pieces can be changed by the end user. Fo this step we will demonstrate use of  both the python api and the cli.

The third step is building functionality for processing and delivery with the Orders API. At the time of the creation of this tutorial (June 2019), the Orders API functionality best supported (e.g. [documented](https://planetlabs.github.io/planet-client-python/cli/examples.html#orders-examples)) in the command-line interface (CLI). Therefore, we will use the client CLI for this portion of the tutorial.

Finally, once we have downloaded the order, we will unzip it and visualize the resulting imagery.


To summarize, these are the steps:
1. [Initialize API client](#Step-1:-Initialize-API-client)
1. [Search Data API](#Step-2:-Search-Data-API)
1. [Submit Order](#Step-3:-Submit-Order)
1. [Download Order](#Step-4:-Download-Order)
1. [Unzip and View Order](#Step-5:-Unzip-and-View-Order)

#### Import Dependencies

In [ ]:
from datetime import datetime
import json
import os
from pathlib import Path
from pprint import pprint
import time
from zipfile import ZipFile
import planet

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from planet import Auth
from planet import Session, DataClient, OrdersClient, data_filter, order_request
import rasterio
from rasterio import plot
from shapely.geometry import MultiPolygon, shape

#### Step 1: Initialize API client

In [ ]:
# if your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.getenv('PL_API_KEY', 'PASTE_YOUR_KEY_HERE')

In [ ]:
client = Auth.from_key(API_KEY)

#### Step 2: Search Data API

The goal of this step is to get the scene ids that meet the search criteria for this use case.

In [ ]:
# define test data for the filter

# iowa crops aoi
test_aoi_geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [-93.299129, 42.699599],
            [-93.299674, 42.812757],
            [-93.288436, 42.861921],
            [-93.265332, 42.924817],
            [-92.993873, 42.925124],
            [-92.993888, 42.773637],
            [-92.998396, 42.754529],
            [-93.019154, 42.699988],
            [-93.299129, 42.699599]
        ]
    ]
}

### Let's search:
# for the geometry above
# Date Range: Jan 1st - Jan 31st 2020
# Clear Percent: 90% or above

In [ ]:
# create an API Request from the search specifications

item_type = ['PSScene']

geom_filter = data_filter.geometry_filter(test_aoi_geom)
clear_percent_filter = data_filter.range_filter('clear_percent', None, None, 90)
date_range_filter = data_filter.date_range_filter("acquired", datetime(month=1, day=1, year=2020), datetime(month=1, day=31, year=2020))

combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter])

async with Session() as sess:
    cl = DataClient(sess)
    request = await cl.create_search(name='temp_search',search_filter=combined_filter, item_types=item_type)


In [ ]:
# Let's look at our search request.
# Note: This is just the request's structure, the search hasn't been implemented yet
request

In [ ]:
# Search the Data API
async with Session() as sess:
    cl = DataClient(sess)
    items = await cl.run_search(search_id=request['id'])
    item_list = [i async for i in items]

In [ ]:
print(len(item_list))

In [ ]:
# check out an item just for fun
print(item_list[0])

In [ ]:
# visualize a scene footprint
footprints = [shape(i['geometry']) for i in item_list]
footprints[0]

In [ ]:
# visualize subset of footprints and aoi
MultiPolygon([shape(test_aoi_geom), *footprints[:5]])

As we can see, the footprints (rectangles) do not exactly match the AOI. Indeed, none of them cover the AOI. We don't care about pixels outside of the AOI, so we are going to want to clip the imagery to the AOI (to remove pixels outside the AOI).

#### Step 3: Submit Order

Now that we have the scene ids, we can create the order. The output of this step is a single zip file that contains all of the scenes that meet our criteria.

Because this is a demo, we don't download all of the scene ids.

In [ ]:
# work with just a subset of the items in the interest of bandwidth
test_items = item_list[:2]

In [ ]:
# filter to item ids
ids = [i['id'] for i in test_items]
ids

In [ ]:
# specify the psscene4band surface reflectance product
# make sure to get the *_udm2 bundle so you get the udm2 product
# note: capitalization really matters in item_type when using planet client orders api
item_type = 'PSScene'
bundle = 'analytic_sr_udm2'
# specify a name
name = 'tutorial_order'

##### Step 3.1: Build Orders Toolchain

In [ ]:
# specify tools

# clip to AOI
clip_tool = {'clip': {'aoi': test_aoi_geom}}

# convert to NDVI
bandmath_tool = {'bandmath': {
    "pixel_type": "32R",
    "b1": "(b4 - b3) / (b4+b3)"
}}

tools = [clip_tool, bandmath_tool]
pprint(tools)

In [ ]:
# We can build our order request using the Python SDK's order_request feature

products = [order_request.product(ids, bundle, item_type)]

request = order_request.build_request('test_order_sdk_method_2', products=products, tools=tools)

In [ ]:
# Let's take a look at our request
request

In [ ]:
# save the request as a json file as well
with open('json_request.txt', 'w') as convert_file:
     convert_file.write(json.dumps(request))

##### Step 3.2: Submit Order

###### Option 1: Client Python API

The first option for submitting an order is using the planet client python api.

In [ ]:
# Place the order
async with Session() as sess:
    cl = OrdersClient(sess)
    order = await cl.create_order(request)

In [ ]:
# View the order info
order

###### Option 2: Client CLI

In some instances, using the CLI to submit orders may be desired.

In [ ]:
!planet orders create 'json_request.txt'

note that we see our Order ID above.

#### Step 4: Download Order

To download the order from the orders api, we will use the planet python client CLI. It would be nice to use the python client python api for this step but, as of the writing of this tutorial, support for the orders api in the planet client python api has been [confusing](https://github.com/planetlabs/planet-client-python/issues/217). The CLI download status output is also [confusing and possibly wrong](https://github.com/planetlabs/planet-client-python/issues/218) but the CLI does easily and successfully download the order.

When we download an order, we always get a `manifest.json` file. Therefore, while we only ordered one file (an order zip), we will download two files. The manifest is [very useful](https://developers.planet.com/docs/orders/ordering-delivery/#why-you-should-depend-on-the-manifest-file) and we will use it to locate the zip file we ordered and downloaded.

##### Step 4.1: Run Download

For this step we will use the Orders CLI because it is the easiest and best supported way to download via the planet client for now.

One thing to watch out for: if you have already downloaded an order, it won't be available for re-download. You will have to resubmit the order and get a new order id. What is tricky is that the manifest is still downloaded, but the other files are not.

In [ ]:
demo_data_dir = os.path.join('data', 'demo')

# make the download directory if it doesn't exist
Path(demo_data_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
# For this example, we will use the Order ID from our API Client request

order_id = order["id"]

In [ ]:
# First, we will make sure the order has reached a downloadable state. This may take several minutes.
!planet orders wait $order_id

In [ ]:
!planet orders download $order_id --directory $demo_data_dir

##### Step 4.2: Get Downloaded File Location(s)

We use the downloaded order manifest to find the downloaded file locations. The manifest is saved in the download directory.

In [ ]:
!ls data/demo

In [ ]:
def get_download_locations(download_dir):
    manifest_file = os.path.join(download_dir, order_id, 'manifest.json')
    with open(manifest_file, 'r') as src:
        manifest = json.load(src)
    
    # uncomment to see the manifest
    # pprint(manifest)
        
    locations = [os.path.join(download_dir, order_id, f['path'])
                 for f in manifest['files']]
    return locations

locations = get_download_locations(demo_data_dir)
pprint(locations)

#### Step 5: Unzip and View Order

In this step we will simply unzip the order and view the downloaded images and their usable data masks.

##### 5.1: Unzip Order

We will unzip the order into a directory named after the file, then we will find the downloaded files (they are in a `files` subdirectory)

In [ ]:
def unzip(filename):
    location = Path(filename)
    
    zipdir = location.parent / location.stem
    with ZipFile(location) as myzip:
        myzip.extractall(zipdir)
    return zipdir

zipdir = unzip(locations[0])
zipdir

In [ ]:
def get_unzipped_files(zipdir):
    filedir = zipdir / 'files'
    filenames = os.listdir(filedir)
    return [filedir / f for f in filenames]

file_paths = get_unzipped_files(zipdir)
pprint(file_paths)

##### 5.2: Visualize Images

In this section we will find the image files and their associated UDMs and we will visualize them.

The first band of the UDM2 file is the clear/not-clear band. 0: not-clear, 1: clear.

In [ ]:
def get_image_and_udm_files(file_paths):
    files = [str(p) for p in file_paths]
    
    # the image files are tiffs and are identified with '_SR_' in the name
    img_id = '_AnalyticMS_SR_'
    imgfiles = [f for f in files
                if f.endswith('.tif') and img_id in f]
    
    # get associated udm files for image files
    # each image has a unique id at the beginning of the name
    imgroots = [str(f).split(img_id)[0] for f in imgfiles]
    
    # the udm files are identified with '_udm2' in the name
    udmfiles = [next(f for f in files if f.startswith(r + '_udm2'))
                for r in imgroots]
    
    return imgfiles, udmfiles

imgfiles, udmfiles = get_image_and_udm_files(file_paths)
pprint(imgfiles)
pprint(udmfiles)

In [ ]:
# read UDM2 file
def read_notclear(udm2_filename):
    with rasterio.open(udm2_filename) as img:
        # the first band is the clear/not clear band
        mask=img.read(1)
        not_clear = mask == 0
        return not_clear
    
udmfile = udmfiles[0]
not_clear = read_notclear(udmfile)

In [ ]:
# there is an issue where some udms aren't the same size as the images
# to deal with this just cut off any trailing rows/columns
# this isn't ideal as it can result in up to one pixel shift in x or y direction
def crop(img, shape):
    return img[:shape[0], :shape[1]]

def read_ndvi(img_filename, not_clear):
    with rasterio.open(imgfile) as img:
        # ndvi is a single-band image
        band = img.read(1)
        
        # crop image and mask to same size
        img_shape = min(band.shape, not_clear.shape)
        ndvi = np.ma.array(crop(band, img_shape), mask=crop(not_clear, img_shape))
    return ndvi
    
imgfile = imgfiles[0]
ndvi = read_ndvi(imgfile, not_clear)

In [ ]:
# set up NDVI visualization
# copied from:  https://stackoverflow.com/a/48598564

"""
The NDVI values will range from -1 to 1. You want to use a diverging color scheme to visualize the data,
and you want to center the colorbar at a defined midpoint. The class below allows you to normalize the colorbar.
"""
class MidpointNormalize(colors.Normalize):
    """
    Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)
    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
    Credit: Joe Kington, http://chris35wills.github.io/matplotlib_diverging_colorbar/
    Credit: https://stackoverflow.com/a/48598564
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)
    
    def __call__(self, value, clip=None):    
        # Note that I'm ignoring clipping and other edge cases here.
        result, is_scalar = self.process_value(value)
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.array(np.interp(value, x, y), mask=result.mask, copy=False)

In [ ]:
def show_ndvi(ndvi):
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)

    # diverging color scheme chosen from https://matplotlib.org/users/colormaps.html
    cmap = plt.cm.RdYlGn 

    mmin = np.nanmin(ndvi)
    mmax = np.nanmax(ndvi)

    mid = 0

    cax = ax.imshow(ndvi, cmap=cmap, clim=(mmin, mmax),
                    norm=MidpointNormalize(midpoint=mid,vmin=mmin, vmax=mmax))

    ax.axis('off')
    ax.set_title('Normalized Difference Vegetation Index', fontsize=18, fontweight='bold')

    cbar = fig.colorbar(cax, orientation='horizontal', shrink=0.65)

    plt.show()

In [ ]:
for imgfile, udmfile in zip(imgfiles, udmfiles):
    show_ndvi(read_ndvi(imgfile, read_notclear(udmfile)))

Okay, we got some beautiful NDVI images down! Notice on the second image that the clouds are masked out thanks to the UDM2 band.